<a href="https://colab.research.google.com/github/MaxVortman/IDEA-code-clones/blob/master/colab/code_clones_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from scipy import sparse
from sklearn.decomposition import TruncatedSVD

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
def resample(X, y):
  rus = RandomUnderSampler(random_state=23)
  return rus.fit_resample(X, y)

## Bag of words

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
!ls 

gdrive	sample_data


In [5]:
%cd ./gdrive/My Drive/code-clones

/content/gdrive/My Drive/code-clones


In [0]:
import metrics

In [0]:
sparse_bow_X = sparse.load_npz("csv/bagofwords_vectors_X.npz")
sparse_bow_y = sparse.load_npz("csv/bagofwords_vectors_y.npz").T

In [8]:
%cd / 

/


In [9]:
sparse_bow_y.shape

(19990, 1)

In [10]:
sparse_bow_X.shape

(19990, 52736)

In [0]:
X_resampled, y_resampled = resample(sparse_bow_X, sparse_bow_y.toarray())
#print(sorted(Counter(y_resampled.T).items()))
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=23)

In [12]:
!git clone https://github.com/hyperopt/hyperopt-sklearn
%cd hyperopt-sklearn
!pip install -e .

Cloning into 'hyperopt-sklearn'...
remote: Enumerating objects: 1176, done.
remote: Total 1176 (delta 0), reused 0 (delta 0), pack-reused 1176
Receiving objects: 100% (1176/1176), 2.00 MiB | 24.66 MiB/s, done.
Resolving deltas: 100% (714/714), done.
/hyperopt-sklearn
Obtaining file:///hyperopt-sklearn
  Running setup.py develop for hpsklearn


In [13]:
from hpsklearn import HyperoptEstimator, any_sparse_classifier, any_classifier
from hyperopt import tpe

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


## Сustom loss

In [0]:
def custom_loss(y_target, y_prediction):
  k_fn = 1
  k_fp = 10
  n = len(y_target)
  (fnr, fpr) = metrics.calc_fnr_fpr(y_target, y_prediction)
  return k_fn * fnr + k_fp * fpr

In [0]:
def hyperopt_custom_loss_estimator(X_train, y_train, sparse=True):
  classifier = any_sparse_classifier('clf') if sparse else any_classifier('clf')
  estim = HyperoptEstimator(classifier=classifier,
                          preprocessing=[],
                          algo=tpe.suggest, 
                          trial_timeout=300,
                          loss_fn=custom_loss,
                          seed=23)
  estim.fit(X_train, y_train)
  return estim

## Custom loss bow

In [16]:
estim = hyperopt_custom_loss_estimator(X_train, y_train)

100%|██████████| 1/1 [00:00<00:00, 18.00it/s, best loss: 1.317986494782075]


In [0]:
y_pred = estim.predict(X_test)

In [18]:
metrics.print_fnr_fpr(y_test, y_pred)


FNR: 
0.13443830570902393

FPR: 
0.13848987108655617


In [19]:
metrics.print_metrics(y_test, y_pred)


accuracy: 
0.7270718232044199

precision: 
0.7302725968436155

recall: 
0.7360809833694866


In [20]:
estim.best_model()

{'ex_preprocs': (),
 'learner': SGDClassifier(alpha=0.06118428364662971, average=False, class_weight=None,
               early_stopping=False, epsilon=0.1, eta0=0.0003970052556398526,
               fit_intercept=True, l1_ratio=0.5022724095962902,
               learning_rate='constant', loss='log', max_iter=158684350.0,
               n_iter_no_change=5, n_jobs=1, penalty='l2',
               power_t=0.907051629874976, random_state=0, shuffle=True,
               tol=0.006308140398304822, validation_fraction=0.1, verbose=False,
               warm_start=False),
 'preprocs': ()}

In [0]:
thr = 0.5
y_prob = [y[1] for y in estim.best_model()['learner'].predict_proba(X_test)]
y_pred = [int(y >= thr) for y in y_prob]
_, fpr = metrics.calc_fnr_fpr(y_test, y_pred)
while fpr - 0.10 >= 0.005:
  thr += 0.0005
  y_pred = [int(y >= thr) for y in y_prob]
  _, fpr = metrics.calc_fnr_fpr(y_test, y_pred)

In [22]:
thr

0.5189999999999979

In [25]:
metrics.print_roc_auc(y_test, y_prob)
metrics.print_fnr_fpr(y_test, y_pred)
metrics.print_metrics(y_test, y_pred)


roc auc: 
0.8184160299127761

FNR: 
0.17642725598526704

FPR: 
0.10497237569060773

accuracy: 
0.7186003683241252

precision: 
0.7603027754415476

recall: 
0.6536514822848879


## Custom loss svd bow

In [0]:
tsvd = TruncatedSVD(n_components=sparse_bow_X.shape[1] // 10, random_state=23, n_iter=10)
tsvd.fit(sparse_bow_X)
tsvd_var_ratios = tsvd.explained_variance_ratio_

In [0]:
# Create a function
def select_n_components(var_ratio, goal_var):
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components, total_variance

In [42]:
select_n_components(tsvd_var_ratios, 0.99)

(65, 0.9900130908094295)

In [43]:
tsvd = TruncatedSVD(n_components=65, random_state=23, n_iter=10)
X = tsvd.fit_transform(sparse_bow_X)
sum(tsvd.explained_variance_ratio_)

0.9900126162151571

In [44]:
X.shape

(19990, 65)

In [0]:
X_resampled, y_resampled = resample(X, sparse_bow_y.toarray())
#print(sorted(Counter(y_resampled.T).items()))
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=23)

In [46]:
estim = hyperopt_custom_loss_estimator(X_train, y_train, sparse=False)

100%|██████████| 1/1 [00:03<00:00,  3.94s/it, best loss: 1.09085328422345]


In [47]:
y_pred = estim.predict(X_test)
metrics.print_fnr_fpr(y_test, y_pred)
metrics.print_metrics(y_test, y_pred)


FNR: 
0.16427255985267036

FPR: 
0.12670349907918968

accuracy: 
0.7090239410681399

precision: 
0.7314597970335676

recall: 
0.6775126536514823


In [48]:
estim.best_model()

{'ex_preprocs': (),
 'learner': GradientBoostingClassifier(criterion='friedman_mse', init=None,
                            learning_rate=0.005891891362147046, loss='deviance',
                            max_depth=None, max_features='log2',
                            max_leaf_nodes=None, min_impurity_decrease=0.0,
                            min_impurity_split=None, min_samples_leaf=8,
                            min_samples_split=2, min_weight_fraction_leaf=0.0,
                            n_estimators=16, n_iter_no_change=None,
                            presort='auto', random_state=0, subsample=1.0,
                            tol=0.0001, validation_fraction=0.1, verbose=0,
                            warm_start=False),
 'preprocs': ()}

In [0]:
thr = 0.5
y_prob = [y[1] for y in estim.best_model()['learner'].predict_proba(X_test)]
y_pred = [int(y >= thr) for y in y_prob]
_, fpr = metrics.calc_fnr_fpr(y_test, y_pred)
while fpr - 0.10 >= 0.005:
  thr += 0.0005
  y_pred = [int(y >= thr) for y in y_prob]
  _, fpr = metrics.calc_fnr_fpr(y_test, y_pred)

In [50]:
thr

0.5034999999999996

In [51]:
metrics.print_roc_auc(y_test, y_prob)
metrics.print_fnr_fpr(y_test, y_pred)
metrics.print_metrics(y_test, y_pred)


roc auc: 
0.8071824536032778

FNR: 
0.20405156537753222

FPR: 
0.10128913443830571

accuracy: 
0.6946593001841621

precision: 
0.7509057971014492

recall: 
0.5994215473608099


## code2vec

In [0]:
df1 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/lwjgl3_code2vec_vectors1.csv")
df2 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/lwjgl3_code2vec_vectors2.csv")
df3 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/lwjgl_code2vec_vectors.csv")
df4 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/spring_code2vec_vectors.csv")
df5 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/jenkins_code2vec_vectors.csv")
df6 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/rxjava_code2vec_vectors.csv")
df_c2v = pd.concat([df1, df2, df3, df4, df5, df6], axis=0)

In [0]:
df_c2v.head()

,exp,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384
0,0.0,0.708866,-0.139283,0.790398,0.880303,0.291317,0.553174,0.823598,-0.693108,-0.379951,-0.536196,-0.057756,-0.763562,-0.781960,-0.448249,-0.533357,-0.451522,-0.065547,0.545325,0.905961,0.846379,0.947172,-0.874068,0.936903,-0.440319,0.009643,-0.558544,0.288042,-0.060373,0.068816,-0.541809,-0.713808,0.880594,-0.628212,-0.872760,0.714413,-0.542334,0.740742,-0.198978,-0.450086,...,0.075166,-0.686472,0.770341,0.886230,0.070973,0.990748,0.556059,-0.979709,0.716640,-0.159755,-0.584313,0.863680,-0.305081,0.005122,0.208050,-0.919549,0.481194,-0.905497,0.054141,0.132897,-0.831053,0.891648,0.524972,-0.797217,0.006505,-0.805470,0.439665,-0.932609,-0.972230,0.444506,0.409421,-0.743124,-0.406415,0.348190,-0.722999,0.883331,0.975385,0.811376,0.529098,-0.580801
1,0.0,0.457570,0.465724,0.049870,0.329546,0.185015,-0.367061,0.646942,0.066054,-0.377213,0.211417,-0.618707,0.051697,-0.098210,0.367054,-0.714161,-0.273630,0.390221,0.230662,0.526845,-0.471009,-0.086963,0.647146,-0.149217,0.850062,0.063569,-0.201275,-0.523791,0.217598,0.393475,-0.242300,0.048727,0.385479,-0.608135,-0.076953,0.053418,0.257176,-0.541758,-0.838411,0.164181,...,0.243302,-0.210837,-0.267095,-0.873269,-0.082603,-0.095414,-0.132618,-0.470093,-0.649802,0.424356,0.059514,-0.261998,-0.033176,0.049180,0.144208,0.056680,0.152013,-0.175859,-0.114169,0.529386,0.405167,0.616029,-0.260005,-0.537304,0.111804,0.100389,0.545154,-0.169261,0.353477,0.224362,0.422311,0.542749,0.726254,0.081205,-0.139942,-0.282454,0.584696,0.316820,0.038821,-0.080841
2,0.0,0.674882,-0.227605,-0.051460,0.322587,0.113033,-0.206298,0.618875,-0.831079,0.467831,0.223961,-0.474329,-0.135923,0.431246,-0.218008,-0.649191,0.012374,0.336949,0.886156,0.454673,-0.639669,0.443773,0.031497,0.345675,0.482762,-0.693035,0.416126,-0.515968,0.172227,0.217237,-0.311850,-0.614717,0.240569,-0.554820,-0.054012,-0.354445,-0.449013,-0.176670,-0.092752,-0.221199,...,-0.175787,-0.184850,-0.531791,-0.455446,-0.020367,0.270404,0.041985,-0.550160,0.767969,0.171719,-0.247429,-0.261586,-0.072140,-0.336018,-0.227835,-0.287362,-0.423872,-0.743692,0.063031,0.646673,-0.507511,0.666606,-0.274423,0.246878,-0.009419,-0.132733,0.432687,0.648385,0.087229,-0.328465,0.380035,-0.358241,0.168523,-0.115633,0.013167,-0.600655,-0.152489,0.309746,0.104927,-0.338480
3,0.0,0.238518,0.239017,-0.329432,0.076081,0.674525,0.240891,-0.138851,0.377708,-0.052835,0.758993,0.212176,0.319090,0.772388,-0.185795,0.141485,-0.263851,-0.114226,0.478966,-0.436920,-0.188591,-0.345383,0.206126,-0.520585,0.897969,0.783976,0.424795,0.498398,-0.557342,0.319309,-0.687786,0.658537,0.692565,0.063520,0.564135,0.445329,-0.448139,-0.636677,0.653752,-0.661554,...,0.373130,-0.401377,-0.644872,0.041106,0.609853,-0.099505,0.522919,-0.703145,0.073218,-0.380949,-0.728012,-0.726868,-0.303032,-0.133776,0.549605,-0.556467,0.017998,-0.530820,-0.106090,-0.760480,-0.038897,0.251707,0.725792,0.145969,0.424751,-0.584596,-0.767587,0.301838,0.746892,-0.455481,-0.629672,-0.818671,-0.240460,0.834071,-0.167629,-0.331171,-0.589584,0.668989,0.260057,0.465806
4,0.0,-0.261068,-0.683279,-0.386828,0.031994,-0.891767,0.202983,-0.022529,0.667178,0.515003,0.826141,-0.176847,-0.039715,-0.733276,0.443135,-0.023443,-0.327092,-0.362199,0.233987,0.319233,0.299280,-0.234696,0.303305,0.170403,-0.224727,0.483681,-0.372073,0.648301,-0.407349,-0.432478,0.606037,-0.137597,0.223075,0.147892,-0.423443,-0.441040,-0.332034,-0.342627,0.125737,-0.407850,...,-0.610979,-0.784399,0.278932,-0.380384,-0.457574,0.915914,0.290403,-0.275704,-0.058648,0.762267,-0.227620,0.422656,0.326875,-0.433874,0.615914,0.239106,-0.050791,-0.410500,-0.014974,0.270726,-0.353269,0.280647,-0.001562,0.455043,0.148662,0.320710,-0.886484,0.124052,0.072454,0.706325,-0.393192,0.125753

In [0]:
y = df_c2v['exp']
X = df_c2v.drop('exp', 1)
X_resampled, y_resampled = resample(X, y)
#print(sorted(Counter(y_resampled.T).items()))
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=23)

## Custom loss code2vec

In [0]:
estim = hyperopt_custom_loss_estimator(X_train, y_train, sparse=False)

100%|██████████| 1/1 [00:04<00:00,  4.02s/it, best loss: 1.8229768786127167]


In [0]:
y_pred = estim.predict(X_test)

In [0]:
metrics.print_fnr_fpr(y_test, y_pred)

(0.07368877329865627, 0.19592544429995665)

In [0]:
metrics.print_metrics(y_test, y_pred)

accuracy: 
0.7303857824013871

precision: 
0.6861111111111111

recall: 
0.853195164075993


In [0]:
estim.best_model()

{'ex_preprocs': (),
 'learner': GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.005891891362147046, loss='deviance',
               max_depth=None, max_features='log2', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=8, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=16,
               n_iter_no_change=None, presort='auto', random_state=0,
               subsample=1.0, tol=0.0001, validation_fraction=0.1,
               verbose=0, warm_start=False),
 'preprocs': ()}

In [0]:
thr = 0.5
y_prob = [y[1] for y in estim.best_model()['learner'].predict_proba(X_test)]
y_pred = [int(y >= thr) for y in y_prob]
_, fpr = metrics.calc_fnr_fpr(y_test, y_pred)
while fpr - 0.10 >= 0.005:
  thr += 0.0005
  y_pred = [int(y >= thr) for y in y_prob]
  _, fpr = metrics.calc_fnr_fpr(y_test, y_pred)

In [0]:
thr

0.5084999999999991

In [0]:
metrics.print_all_metrics(y_test, y_prob)


roc auc: 
0.8261355147000243
